In [47]:
import pandas as pd
import tensorflow as tf


In [48]:
class ColumnNames(object):
    """
    Column names
    """

    def __init__(self, user_col, item_col, rating_col):
        self.user_col = user_col
        self.item_col = item_col
        self.rating_col = rating_col

In [49]:
class NcfTrainingArguments(object):
    def __init__(self, user_dim, item_dim, batch_size, num_epochs, hidden1_dim,
                 hidden2_dim):
        self.user_dim = user_dim
        self.item_dim = item_dim
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.hidden1_dim = hidden1_dim
        self.hidden2_dim = hidden2_dim


config = {
    'training_args':
    NcfTrainingArguments(
        user_dim=10,
        item_dim=6,
        batch_size=16,
        num_epochs=5,
        hidden1_dim=8,
        hidden2_dim=2),
    'col_names':
    ColumnNames(user_col='user', item_col='item', rating_col='rating')
}

In [50]:
class MatrixFactorization(object):
    """
    This class implements matrix factorization using the tf2 api
    """

    def __init__(self, n_users, n_items, user_dim, item_dim):
        self.n_users = n_users
        self.n_items = n_items
        self.user_dim = user_dim
        self.item_dim = item_dim
        self.input_user, self.input_item, self.input_rating, self.user_embeddings, self.item_embeddings =\
            self.inputs_weights_init()

    @staticmethod
    def inputs_init():
        """
        Initialises the necessary inputs
        :return:
        """
        input_user = tf.keras.Input((1, ))
        input_item = tf.keras.Input((1, ))
        input_rating = tf.keras.Input((1, ))
        return input_user, input_item, input_rating

    def embeddings_layers_init(self):
        """
        Initialises the embeddings layers
        :return:
        """

        user_embeddings = tf.keras.layers.Embedding(
            self.n_users, self.user_dim, input_length=1)

        item_embeddings = tf.keras.layers.Embedding(
            self.n_items, self.item_dim, input_length=1)

        return user_embeddings, item_embeddings

    def inputs_weights_init(self):
        """
        Initialises inputs and weights
        :return:
        """
        input_user, input_item, input_rating = self.inputs_init()
        user_embeddings, item_embeddings = self.embeddings_layers_init()

        return input_user, input_item, input_rating, user_embeddings, item_embeddings

    def predict(self):
        """
        Generate predictions by defining the architecture
        :return:
        """
        input_item_vector = self.item_embeddings(self.input_item)
        input_user_vector = self.user_embeddings(self.input_user)
        input_item_vector_reshaped = tf.keras.layers.Reshape(
            (self.item_dim, 1))(input_item_vector)
        input_user_vector_reshaped = tf.keras.layers.Reshape(
            (self.user_dim, 1))(input_user_vector)

        dot_product = tf.keras.layers.dot(
            [input_item_vector_reshaped, input_user_vector_reshaped], axes=2)
        predicted_rating = tf.keras.layers.Dense(
            1, activation='linear')(dot_product)
        return predicted_rating

    def model(self):
        """
        Define the keras model
        :return:
        """
        model = tf.keras.Model(
            inputs=[self.input_user, self.input_item], outputs=self.predict())
        return model

In [51]:
class NeuralCF(object):
    """
    This class implements matrix factorization using the tf2 api
    """

    def __init__(self, n_users, n_items, user_dim, item_dim, hidden1_dim,
                 hidden2_dim):
        self.n_users = n_users
        self.n_items = n_items
        self.user_dim = user_dim
        self.item_dim = item_dim
        self.hidden1_dim = hidden1_dim
        self.hidden2_dim = hidden2_dim
        self.input_user, self.input_item, self.input_rating, self.user_embeddings, self.item_embeddings =\
            self.inputs_weights_init()

    @staticmethod
    def inputs_init():
        """
        Initialises the necessary inputs
        :return:
        """
        input_user = tf.keras.Input((1, ))
        input_item = tf.keras.Input((1, ))
        input_rating = tf.keras.Input((1, ))
        return input_user, input_item, input_rating

    def embeddings_layers_init(self):
        """
        Initialises the embeddings layers
        :return:
        """

        user_embeddings = tf.keras.layers.Embedding(
            self.n_users, self.user_dim, input_length=1)

        item_embeddings = tf.keras.layers.Embedding(
            self.n_items, self.item_dim, input_length=1)

        return user_embeddings, item_embeddings

    def inputs_weights_init(self):
        """
        Initialises inputs and weights
        :return:
        """
        input_user, input_item, input_rating = self.inputs_init()
        user_embeddings, item_embeddings = self.embeddings_layers_init()

        return input_user, input_item, input_rating, user_embeddings, item_embeddings

    def predict(self):
        """
        Generate predictions by defining the architecture
        :return:
        """
        input_item_vector = self.item_embeddings(self.input_item)
        input_user_vector = self.user_embeddings(self.input_user)
        input_item_vector_reshaped = tf.keras.layers.Reshape(
            (self.item_dim, 1))(input_item_vector)
        input_user_vector_reshaped = tf.keras.layers.Reshape(
            (self.user_dim, 1))(input_user_vector)

        # concatenation of user and item embeddings
        user_item_vector_concat = tf.keras.layers.concatenate(
            [input_item_vector_reshaped, input_user_vector_reshaped], axis=1)

        # first dense layer
        dense1 = tf.keras.layers.Dense(
            self.hidden1_dim)(user_item_vector_concat)
        dropout_1 = tf.keras.layers.Dropout(0.1)(dense1)
        # second dense layer
        dense2 = tf.keras.layers.Dense(self.hidden2_dim)(dropout_1)
        predicted_rating = tf.keras.layers.Dense(
            1, activation='linear')(dense2)
        return predicted_rating

    def model(self):
        """
        Define the keras model
        :return:
        """
        model = tf.keras.Model(
            inputs=[self.input_user, self.input_item], outputs=self.predict())
        return model


In [52]:
class ColumnIndexer(object):
    """
    This class in used in order to index
    columns like user id, item id in a pandas dataframe
    """

    def __init__(self, df, col_names):
        self.df = df
        self.col_names = col_names
        self.distinct_items = self.get_distinct_items()
        self.indexers, self.reverse_indexers = self.generate_indexers()

    def get_distinct_items(self):
        """
        Get all the distinct item in the column
        :return:
        """
        distinct_items = {
            col_name: set(self.df[col_name].values)
            for col_name in self.col_names
        }
        return distinct_items

    def generate_indexers(self):
        """
        Builds the indexers and the reverse indexers for all
        the columns defined when instantiating the class
        :return:
        """
        indexers = {
            col: {k: v
                  for v, k in enumerate(distinct_item)}
            for col, distinct_item in self.distinct_items.items()
        }
        reverse_indexers = {
            col: {v: k
                  for k, v in indexer.items()}
            for col, indexer in indexers.items()
        }
        return indexers, reverse_indexers

    def transform(self, dataset):
        """                                                                     
        Transforms the original dataset
        by adding indexed versions of the columns
        :param dataset: Original pandas dataframe
        :return:
        """
        for col in self.col_names:
            dataset[col + '_indexed'] = dataset[col]\
                .map(self.indexers[col])
        return dataset

In [53]:
class ItemSimilarityCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        None

In [54]:
class MfTrainingArguments(object):
    """
    Training arguments
    """

    def __init__(self, user_dim, item_dim, batch_size, num_epochs):
        self.user_dim = user_dim
        self.item_dim = item_dim
        self.batch_size = batch_size
        self.num_epochs = num_epochs


config = {
    'training_args':
    MfTrainingArguments(user_dim=10, item_dim=5, batch_size=16, num_epochs=5),
    'col_names':                                               
    ColumnNames(user_col='user', item_col='item', rating_col='r                       ting')
}

In [55]:
def main(config):
    training_args = config['training_args']
    col_names = config['col_names']                                                                                                                     

    # read in the train set
    train = pd.read_csv(                                                                                        
        'tf2recommender/data/u1_base.csv',
        sep='\t',
        header=None,
        usecols=[0, 1, 2],
        names=[col_names.user_col, col_names.item_col, col_names.rating_col])

    # read in the test set
    test = pd.read_csv(
        'tf2recommender/data/u1_test.csv',
        sep='\t',
        header=None,
        usecols=[0, 1, 2],
        names=[col_names.user_col, col_names.item_col, col_names.rating_col])

    # drop users and items that do not exist in the training set
    test = test[test['item'].isin(train['item'].values)]

    # instantiate the column index for both user and items
    indexer = ColumnIndexer(train, ['user', 'item'])

    # index the train set
    train = indexer.transform(train)
    # index the test set
    test = indexer.transform(test)

    # get the number of distinct users and items
    number_of_users = len(set(train[col_names.user_col].values))
    number_of_items = len(set(train[col_names.item_col].values))

    # create user item rating tuples
    train_users_items_ratings = ((
        train[col_names.user_col + '_indexed'].values,
        train[col_names.item_col + '_indexed'].values),
                                 train[col_names.rating_col].values)

    test_users_items_ratings = ((test[col_names.user_col + '_indexed'].values,
                                 test[col_names.item_col + '_indexed'].values),
                                test[col_names.rating_col].values)

    # instantiate the tf datasets
    train_dataset = tf.data.Dataset.from_tensor_slices(
        train_users_items_ratings)
    test_dataset = tf.data.Dataset.from_tensor_slices(test_users_items_ratings)

    train_batches = train_dataset.shuffle(1000).batch(training_args.batch_size).prefetch(1)
    test_batches = test_dataset.batch(training_args.batch_size).prefetch(1)

    mf = MatrixFactorization(number_of_users, number_of_items,
                             training_args.user_dim, training_args.item_dim)

    model = mf.model()

    print(model.summary())

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

    model.fit(train_batches, epochs=training_args.num_epochs)

    print('\n# Evaluate')
    print(model.evaluate(test_batches))

In [56]:
class NcfTrainingArguments(object):
    def __init__(self, user_dim, item_dim, batch_size, num_epochs, hidden1_dim,
                 hidden2_dim):
        self.user_dim = user_dim
        self.item_dim = item_dim
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.hidden1_dim = hidden1_dim
        self.hidden2_dim = hidden2_dim


config = {
    'training_args':
    NcfTrainingArguments(
        user_dim=10,
        item_dim=6,
        batch_size=16,
        num_epochs=5,
        hidden1_dim=8,
        hidden2_dim=2),
    'col_names':
    ColumnNames(user_col='user', item_col='item', rating_col='rating')
}

In [57]:
training_args = config['training_args']
col_names = config['col_names']

In [58]:
def main(config):
    training_args = config['training_args']
    col_names = config['col_names']

    # read in the train set
    train = pd.read_csv(
        'tf2recommender/data/u1_base.csv',
        sep='\t',
        header=None,
        usecols=[0, 1, 2],
        names=[col_names.user_col, col_names.item_col, col_names.rating_col])

    # read in the test set
    test = pd.read_csv(
        'tf2recommender/data/u1_test.csv',
        sep='\t',
        header=None,
        usecols=[0, 1, 2],
        names=[col_names.user_col, col_names.item_col, col_names.rating_col])

    # drop users and items that do not exist in the training set
    test = test[test['item'].isin(train['item'].values)]

    # instantiate the column index for both user and items
    indexer = ColumnIndexer(train, ['user', 'item'])

    # index the train set
    train = indexer.transform(train)
    # index the test set
    test = indexer.transform(test)

    # get the number of distinct users and items
    number_of_users = len(set(train[col_names.user_col].values))
    number_of_items = len(set(train[col_names.item_col].values))

    # create user item rating tuples
    train_users_items_ratings = ((
        train[col_names.user_col + '_indexed'].values,
        train[col_names.item_col + '_indexed'].values),
                                 train[col_names.rating_col].values)

    test_users_items_ratings = ((test[col_names.user_col + '_indexed'].values,
                                 test[col_names.item_col + '_indexed'].values),
                                test[col_names.rating_col].values)

    # instantiate the tf datasets
    train_dataset = tf.data.Dataset.from_tensor_slices(
        train_users_items_ratings)
    test_dataset = tf.data.Dataset.from_tensor_slices(test_users_items_ratings)

    train_batches = train_dataset.shuffle(1000).batch(training_args.batch_size).prefetch(1)
    test_batches = test_dataset.batch(training_args.batch_size).prefetch(1)

    ncf = NeuralCF(number_of_users, number_of_items, training_args.user_dim,
                   training_args.item_dim, training_args.hidden1_dim,
                   training_args.hidden2_dim)

    model = ncf.model()

    print(model.summary())

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

    model.fit(train_batches, epochs=training_args.num_epochs)

    print('\n# Evaluate')
    print(model.evaluate(test_batches))

In [59]:
train_args = config['col_names']
col_names = config['col_names']

In [60]:
train = pd.read_csv(
        'data/u1_base.csv',
        sep='\t',
        header=None,
        usecols=[0, 1, 2],
        names=[col_names.user_col, col_names.item_col, col_names.rating_col])

In [61]:
train.head()

,user,item,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


In [62]:
test = pd.read_csv(
        'data/u1_test.csv',
        sep='\t',
        header=None,
        usecols=[0, 1, 2],
        names=[col_names.user_col, col_names.item_col, col_names.rating_col])


In [63]:
# drop users and items that do not exist in the training set
test = test[test['item'].isin(train['item'].values)]

In [64]:
test.head()

,user,item,rating
0,1,6,5
1,1,10,3
2,1,12,5
3,1,14,5
4,1,17,3


In [65]:
# instantiate the column index for both user and items
indexer = ColumnIndexer(train, ['user', 'item'])

In [66]:
# index the train set
train = indexer.transform(train)
# index the test set
test = indexer.transform(test)

In [67]:
train.head()

,user,item,rating,user_indexed,item_indexed
0,1,1,5,0,0
1,1,2,3,0,1
2,1,3,4,0,2
3,1,4,3,0,3
4,1,5,3,0,4


In [68]:
test.head()

,user,item,rating,user_indexed,item_indexed
0,1,6,5,0,5
1,1,10,3,0,9
2,1,12,5,0,11
3,1,14,5,0,13
4,1,17,3,0,16


In [69]:
# get the number of distinct users and items
number_of_users = len(set(train[col_names.user_col].values))
number_of_items = len(set(train[col_names.item_col].values))

In [70]:
print(number_of_users)
print(number_of_items)

943
1650


In [71]:
# create user item rating tuples
train_users_items_ratings = ((
        train[col_names.user_col + '_indexed'].values,
        train[col_names.item_col + '_indexed'].values),
                                 train[col_names.rating_col].values)

test_users_items_ratings = ((test[col_names.user_col + '_indexed'].values,
                                 test[col_names.item_col + '_indexed'].values),
                                test[col_names.rating_col].values)

In [72]:
print(train_users_items_ratings)

((array([  0,   0,   0, ..., 942, 942, 942]), array([   0,    1,    2, ..., 1180, 1220, 1318])), array([5, 3, 4, ..., 3, 3, 3]))


In [73]:
print(test_users_items_ratings)


((array([  0,   0,   0, ..., 458, 459, 461]), array([  5,   9,  11, ..., 927,   9, 680])), array([5, 3, 5, ..., 3, 3, 5]))


In [74]:
# instantiate the tf datasets
train_dataset = tf.data.Dataset.from_tensor_slices(
        train_users_items_ratings)
test_dataset = tf.data.Dataset.from_tensor_slices(test_users_items_ratings)

train_batches = train_dataset.shuffle(1000).batch(training_args.batch_size).prefetch(1)
test_batches = test_dataset.batch(training_args.batch_size).prefetch(1)

ncf = NeuralCF(number_of_users, number_of_items, training_args.user_dim,
                   training_args.item_dim, training_args.hidden1_dim,
                   training_args.hidden2_dim)
model = ncf.model()

print(model.summary())

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

model.fit(train_batches, epochs=training_args.num_epochs)

print('\n# Evaluate')
print(model.evaluate(test_batches))

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 1, 6)         9900        input_11[0][0]                   
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1, 10)        9430        input_10[0][0]                   
____________________________________________________________________________________________

In [75]:
class NeuralCF(object):
    """
    This class implements matrix factorization using the tf2 api
    """

    def __init__(self, n_users, n_items, user_dim, item_dim, hidden1_dim,
                 hidden2_dim):
        self.n_users = n_users
        self.n_items = n_items
        self.user_dim = user_dim
        self.item_dim = item_dim
        self.hidden1_dim = hidden1_dim
        self.hidden2_dim = hidden2_dim
        self.input_user, self.input_item, self.input_rating, self.user_embeddings, self.item_embeddings =\
            self.inputs_weights_init()

    @staticmethod
    def inputs_init():
        """
        Initialises the necessary inputs
        :return:
        """
        input_user = tf.keras.Input((1, ))
        input_item = tf.keras.Input((1, ))
        input_rating = tf.keras.Input((1, ))
        return input_user, input_item, input_rating

    def embeddings_layers_init(self):
        """
        Initialises the embeddings layers
        :return:
        """

        user_embeddings = tf.keras.layers.Embedding(
            self.n_users, self.user_dim, input_length=1)

        item_embeddings = tf.keras.layers.Embedding(
            self.n_items, self.item_dim, input_length=1)

        return user_embeddings, item_embeddings

    def inputs_weights_init(self):
        """
        Initialises inputs and weights
        :return:
        """
        input_user, input_item, input_rating = self.inputs_init()
        user_embeddings, item_embeddings = self.embeddings_layers_init()

        return input_user, input_item, input_rating, user_embeddings, item_embeddings

    def predict(self):
        """
        Generate predictions by defining the architecture
        :return:
        """
        input_item_vector = self.item_embeddings(self.input_item)
        input_user_vector = self.user_embeddings(self.input_user)
        input_item_vector_reshaped = tf.keras.layers.Reshape(
            (self.item_dim, 1))(input_item_vector)
        input_user_vector_reshaped = tf.keras.layers.Reshape(
            (self.user_dim, 1))(input_user_vector)

        # concatenation of user and item embeddings
        user_item_vector_concat = tf.keras.layers.concatenate(
            [input_item_vector_reshaped, input_user_vector_reshaped], axis=1)

        # first dense layer
        dense1 = tf.keras.layers.Dense(
            self.hidden1_dim)(user_item_vector_concat)
        dropout_1 = tf.keras.layers.Dropout(0.1)(dense1)
        # second dense layer
        dense2 = tf.keras.layers.Dense(self.hidden2_dim)(dropout_1)
        predicted_rating = tf.keras.layers.Dense(
            1, activation='linear')(dense2)
        return predicted_rating

    def model(self):
        """
        Define the keras model
        :return:
        """
        model = tf.keras.Model(
            inputs=[self.input_user, self.input_item], outputs=self.predict())
        return model

In [76]:
def main(config):
    training_args = config['training_args']
    col_names = config['col_names']

    # read in the train set
    train = pd.read_csv(
        'tf2recommender/data/u1_base.csv',
        sep='\t',
        header=None,
        usecols=[0, 1, 2],
        names=[col_names.user_col, col_names.item_col, col_names.rating_col])

    # read in the test set
    test = pd.read_csv(
        'tf2recommender/data/u1_test.csv',
        sep='\t',
        header=None,
        usecols=[0, 1, 2],
        names=[col_names.user_col, col_names.item_col, col_names.rating_col])

    # drop users and items that do not exist in the training set
    test = test[test['item'].isin(train['item'].values)]

    # instantiate the column index for both user and items
    indexer = ColumnIndexer(train, ['user', 'item'])

    # index the train set
    train = indexer.transform(train)
    # index the test set
    test = indexer.transform(test)

    # get the number of distinct users and items
    number_of_users = len(set(train[col_names.user_col].values))
    number_of_items = len(set(train[col_names.item_col].values))

    # create user item rating tuples
    train_users_items_ratings = ((
        train[col_names.user_col + '_indexed'].values,
        train[col_names.item_col + '_indexed'].values),
                                 train[col_names.rating_col].values)

    test_users_items_ratings = ((test[col_names.user_col + '_indexed'].values,
                                 test[col_names.item_col + '_indexed'].values),
                                test[col_names.rating_col].values)

    # instantiate the tf datasets
    train_dataset = tf.data.Dataset.from_tensor_slices(
        train_users_items_ratings)
    test_dataset = tf.data.Dataset.from_tensor_slices(test_users_items_ratings)

    train_batches = train_dataset.shuffle(1000).batch(training_args.batch_size).prefetch(1)
    test_batches = test_dataset.batch(training_args.batch_size).prefetch(1)

    mf = MatrixFactorization(number_of_users, number_of_items,
                             training_args.user_dim, training_args.item_dim)

    model = mf.model()

    print(model.summary())

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

    model.fit(train_batches, epochs=training_args.num_epochs)

    print('\n# Evaluate')
    print(model.evaluate(test_batches))

In [77]:
# get the number of distinct users and items
number_of_users = len(set(train[col_names.user_col].values))
number_of_items = len(set(train[col_names.item_col].values))

    # create user item rating tuples
train_users_items_ratings = ((
        train[col_names.user_col + '_indexed'].values,
        train[col_names.item_col + '_indexed'].values),
                                 train[col_names.rating_col].values)

test_users_items_ratings = ((test[col_names.user_col + '_indexed'].values,
                                 test[col_names.item_col + '_indexed'].values),
                                test[col_names.rating_col].values)

    # instantiate the tf datasets
train_dataset = tf.data.Dataset.from_tensor_slices(
        train_users_items_ratings)
test_dataset = tf.data.Dataset.from_tensor_slices(test_users_items_ratings)

train_batches = train_dataset.shuffle(1000).batch(training_args.batch_size).prefetch(1)
test_batches = test_dataset.batch(training_args.batch_size).prefetch(1)

mf = MatrixFactorization(number_of_users, number_of_items,
                             training_args.user_dim, training_args.item_dim)

model = mf.model()

print(model.summary())

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

model.fit(train_batches, epochs=training_args.num_epochs)

print('\n# Evaluate')
print(model.evaluate(test_batches))

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 1, 6)         9900        input_14[0][0]                   
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 1, 10)        9430        input_13[0][0]                   
____________________________________________________________________________________________

In [42]:
import numpy as np
from keras.layers import Embedding, Reshape
from keras.models import Sequential
from keras.layers import dot

In [43]:
class CFModel(Sequential):

    # The constructor for the class
    def __init__(self, n_users, m_items, k_factors, **kwargs):
        # P is the embedding layer that creates an User by latent factors matrix.
        # If the intput is a user_id, P returns the latent factor vector for that user.
        P = Sequential()
        P.add(Embedding(n_users, k_factors, input_length=1))
        P.add(Reshape((k_factors,)))

        # Q is the embedding layer that creates a Movie by latent factors matrix.
        # If the input is a movie_id, Q returns the latent factor vector for that movie.
        Q = Sequential()
        Q.add(Embedding(m_items, k_factors, input_length=1))
        Q.add(Reshape((k_factors,)))

        super(CFModel, self).__init__(**kwargs)
        
        # The Merge layer takes the dot product of user and movie latent factor vectors to return the corresponding rating.
        self.dot([P, Q], axes=1)
        

    # The rate function to predict user's rating of unrated items
    def rate(self, user_id, item_id):
        return self.predict([np.array([user_id]), np.array([item_id])])[0][0]


In [45]:
print(train_dataset)

<TensorSliceDataset shapes: (((), ()), ()), types: ((tf.int64, tf.int64), tf.int64)>
